In [1]:
import pandas as pd
import os
import awswrangler as wr
env = os.environ.get('Environment')
print(f"Environment: {env!r}")


Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

In [3]:
%%athena_to_df --database f"{env}-luna-pharma-prepared-data" --out df_test 
select 	jh.job_name, jh.sw_job_type,sa.recd_date, sa.c_factor, sa.time_factor, 
		sa.status as sample_status, sa.sw_date_creation,sa.id_text,
   
     pds.due_login_date
from job_header jh 

join sample sa on sa.job_name = jh.job_name
join  pds_study_pull pds on sa.id_text=pds.sample_id_text

where jh.group_id like 'ARA%'
and jh.job_status <> 'A'
and jh.sw_lot_validation = 'F'
and sa.status <> 'U' and sa.status <> 'A'
and from_iso8601_timestamp(sa.recd_date) BETWEEN current_date - INTERVAL '1' day AND current_date
and jh.sw_no_lot not like '%FOR%'
and jh.sw_job_type = 'SOG';


In [4]:
df_test.head(15)

,job_name,sw_job_type,recd_date,c_factor,time_factor,sample_status,sw_date_creation,id_text,due_login_date
0,ARA_362671_2023_01,SOG,2024-06-12T15:14:02,ARA_-20_RUCO,ARA_9M,V,2024-06-12T15:10:40,S-ARA_362671_2023_01-475,2024-06-12T15:10:42
1,ARA_362671_2023_01,SOG,2024-06-12T14:07:23,ARA_2-8DEGRE,ARA_9M,V,2024-06-08T00:02:52,S-ARA_362671_2023_01-476,2024-06-08T00:02:54
2,ARA_362671PE_2023_01,SOG,2024-06-12T14:43:34,ARA_2-8DEGRE,ARA_9M,V,2024-06-08T00:04:33,S-ARA_362671PE_2023_01-488,2024-06-08T00:04:34
3,ARA_362671PE_2023_01,SOG,2024-06-12T14:27:37,ARA_-20_RUCO,ARA_9M,V,2024-06-08T00:04:28,S-ARA_362671PE_2023_01-487,2024-06-08T00:04:29


In [5]:
database_name = f'prod-dmdg-workspace-public-aramon-smartqc-sog-20221021125055926'

In [6]:
table_name = f'{env}_ARA_luna_smartqc_sog'

In [7]:
path = f's3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-sog-20221021125055926'

In [8]:
workspace_prefix =f'public-aramon-smartqc-sog-20221021125055926'

In [9]:
wr.s3.to_parquet(
    df=df_test,
    path=path,
    database=database_name,
    dataset=True,
    table=table_name,
    mode='overwrite',
    index=False,
    compression=None,
)

{'paths': ['s3://sanofi-datalake-dmdg-workspaces-prod/public-aramon-smartqc-sog-20221021125055926/b0fff926487945399fdb47eb5785f767.parquet'],
 'partitions_values': {}}

In [10]:
df_test.shape

(4, 9)